In [5]:
import csv
from typing import List, Tuple
from sqlalchemy import create_engine, Column, Integer, String, Float, select, func
from sqlalchemy.orm import declarative_base, Session

# ---------- ORM-модель ----------
Base = declarative_base()

class Student(Base):
    __tablename__ = "students"
    id         = Column(Integer, primary_key=True, autoincrement=True)
    last_name  = Column(String, nullable=False)
    first_name = Column(String, nullable=False)
    faculty    = Column(String, nullable=False)
    course     = Column(String, nullable=False)
    grade      = Column(Float,  nullable=False)

# ---------- Инициализация БД ----------
engine = create_engine("sqlite:///students.db", future=True, echo=False)
Base.metadata.drop_all(engine)
Base.metadata.create_all(engine)

# ---------- Загрузка из CSV ----------
def bulk_insert_from_csv(csv_path: str) -> int:
    """
    Ожидаемые заголовки: Фамилия, Имя, Факультет, Курс, Оценка
    Возвращает количество добавленных записей.
    """
    added = 0
    with open(csv_path, "r", encoding="utf-8-sig", newline="") as f, Session(engine) as s:
        reader = csv.DictReader(f)
        required = ["Фамилия", "Имя", "Факультет", "Курс", "Оценка"]
        if any(col not in reader.fieldnames for col in required):
            missing = [c for c in required if c not in (reader.fieldnames or [])]
            raise ValueError(f"В CSV нет колонок: {missing}")

        for row in reader:
            s.add(Student(
                last_name  = row["Фамилия"].strip(),
                first_name = row["Имя"].strip(),
                faculty    = row["Факультет"].strip(),
                course     = row["Курс"].strip(),
                grade      = float(row["Оценка"]),
            ))
            added += 1
        s.commit()
    return added

# ---------- Запросы (SELECT) ----------
def students_by_faculty(faculty: str) -> List[Student]:
    with Session(engine) as s:
        return (
            s.query(Student)
             .filter(Student.faculty == faculty)
             .order_by(Student.last_name, Student.first_name)
             .all()
        )

def unique_courses() -> List[str]:
    with Session(engine) as s:
        rows = s.execute(select(func.distinct(Student.course)).order_by(Student.course)).all()
        return [r[0] for r in rows]

def average_grade_by_faculty() -> List[Tuple[str, float]]:
    with Session(engine) as s:
        rows = s.execute(
            select(Student.faculty, func.avg(Student.grade))
            .group_by(Student.faculty)
            .order_by(Student.faculty)
        ).all()
        return [(fac, float(avg)) for fac, avg in rows]

def students_below_grade_in_course(course: str, threshold: float = 30.0) -> List[Student]:
    with Session(engine) as s:
        return (
            s.query(Student)
             .filter(Student.course == course, Student.grade < threshold)
             .order_by(Student.grade.asc())
             .all()
        )

# ---------- Демонстрация ----------
if __name__ == "__main__":
    # 1) Загрузка CSV
    count = bulk_insert_from_csv("students.csv")
    print(f"Загружено записей: {count}\n")

    # 2) Список студентов по факультету
    fac = "РЭФ"
    print(f"Студенты факультета {fac}:")
    for s in students_by_faculty(fac):
        print(f"- {s.last_name} {s.first_name}: {s.course}, {s.grade}")
    print()

    # 3) Уникальные курсы
    print("Уникальные курсы:")
    for c in unique_courses():
        print(f"- {c}")
    print()

    # 4) Средний балл по факультетам
    print("Средний балл по факультетам:")
    for fac, avg in average_grade_by_faculty():
        print(f"- {fac}: {avg:.2f}")
    print()

    # 5) Студенты выбранного курса с оценкой < 30
    course = "Теор. Механика"
    print(f"{course}: студенты с оценкой < 30:")
    for s in students_below_grade_in_course(course, 30.0):
        print(f"- {s.last_name} {s.first_name}: {s.grade}")


Загружено записей: 215

Студенты факультета РЭФ:
- Браун Николай: Психология, 79.0
- Браун Николай: Мат. Анализ, 92.0
- Браун Николай: Теор. Механика, 2.0
- Ву Дмитрий: Теор. Механика, 100.0
- Ву Дмитрий: Информатика, 70.0
- Ву Дмитрий: Теор. Механика, 88.0
- Ву Дмитрий: История, 44.0
- Ву Дмитрий: Мат. Анализ, 3.0
- Джонс Андрей: Мат. Анализ, 4.0
- Джонс Андрей: Информатика, 40.0
- Джонс Андрей: История, 27.0
- Джонс Андрей: Теор. Механика, 57.0
- Джонс Андрей: Психология, 51.0
- Ким Петр: Физика, 56.0
- Ким Петр: Мат. Анализ, 17.0
- Ким Петр: История, 46.0
- Ким Петр: Теор. Механика, 48.0
- Райт Вероника: Информатика, 43.0
- Смит Федор: История, 85.0
- Смит Федор: Физика, 50.0
- Смит Федор: Теор. Механика, 24.0
- Чжоу Мария: Психология, 95.0

Уникальные курсы:
- Информатика
- История
- Мат. Анализ
- Психология
- Теор. Механика
- Физика

Средний балл по факультетам:
- АВТФ: 56.84
- РЭФ: 50.95
- ФЛА: 42.72
- ФПМИ: 49.23
- ФТФ: 49.73

Теор. Механика: студенты с оценкой < 30:
- Райт Веро